### Introspecting a function

In [1]:
def function_wrapper(wrapped):
    def _wrapper(*args, **kwargs):
        return wrapped(*args, **kwargs)
    return _wrapper

@function_wrapper
def function():
    pass

In [2]:
function.__name__

In [3]:
class function_wrapper(object):
    def __init__(self, wrapped):
        self.wrapped = wrapped
    def __call__(self, *args, **kwargs):
        return self.wrapped(*args, **kwargs) 

@function_wrapper
def function():
    pass

In [4]:
function.__name__

AttributeError: 'function_wrapper' object has no attribute '__name__'

Copy the attributes of interest from the wrapped function to the nested wrapper function.

In [5]:
def function_wrapper(wrapped):
    def _wrapper(*args, **kwargs):
        return wrapped(*args, **kwargs)
    _wrapper.__name__ = wrapped.__name__
    _wrapper.__doc__ = wrapped.__doc__
    return _wrapper 

@function_wrapper
def function():
    pass 

In [6]:
function.__name__

'function'

In order to copy the `__module__` attribute and the `__qualname__` and `__annotations__` in Python 3, using `functools.wraps()` in the Python standard library.

In [7]:
import functools 

def function_wrapper(wrapped):
    @functools.wraps(wrapped)
    def _wrapper(*args, **kwargs):
        return wrapped(*args, **kwargs)
    return _wrapper 

@function_wrapper
def function():
    pass 

In [8]:
function.__name__

'function'

If using a class to implement the wrapper, use `functools.update_wrapper()`.

In [9]:
import functools

class function_wrapper(object):
    def __init__(self, wrapped):
        self.wrapped = wrapped
        functools.update_wrapper(self, wrapped)
    def __call__(self, *args, **kwargs):
        return self.wrapped(*args, **kwargs)

当需要参数说明（argument specification）的时候，函数或者类来实现的装饰器都不能满足功能。

In [10]:
import inspect

@function_wrapper
def function():
    pass 

inspect.getfullargspec(function)

FullArgSpec(args=['self'], varargs='args', varkw='kwargs', defaults=None, kwonlyargs=[], kwonlydefaults=None, annotations={})

What's worse, 当使用基于类的装饰器时，会产生被装饰的函数并不是一个函数的异常，而且使用`inspect.getsource()`的时候也会失败。

综上，传统上的装饰器实现并不能满足保留如下功能：
- 保留函数的`__name__`和`__doc__`
- 保留参数说明
- 保留得到函数源代码的能力
- 应用在其他装饰器之上的实现为描述符（descriptor）的装饰器

### Wrapping class methods

In [11]:
class Class(object): 

    @function_wrapper
    def method(self):
        pass 

    @classmethod
    def cmethod(cls):
        pass 

    @staticmethod
    def smethod():
        pass

通常来说一个装饰器能用来包装类中的方法，然而当使用`functools.wraps()`或者`functools.update_wrapper()`来装饰`@classmethod`或者`@staticmethod`时，会抛出异常，因为这些装饰函数并没有能够被复制的某些属性。

In [16]:
class Class(object):
    @function_wrapper
    @classmethod
    def cmethod(cls):
        pass

上述代码在Python 2中会报错，而在Python 3中则没有，因为Python 3中的实现将这些丢失的属性忽略了。

In [17]:
Class.cmethod()

TypeError: 'classmethod' object is not callable

然而，即使我们使用Python 3，依然会产生一些问题，因为这两种装饰函数都基于如下假设：被装饰的函数是被直接调用的。实际上并非如此，我们可以将被装饰的函数可以被称作所谓的描述符，这意味着，若要返回一个能被调用的对象，首先此描述符得正确地绑定给此实例。